In [ ]:
# Pour optimiser les performances de l'analyse de sentiment, il est recommandé de changer le runtime de Colab.
# Voici les étapes pour le faire :
# 1. Accédez au menu "Runtime" (ou "Exécution" en français).
# 2. Cliquez sur "Change runtime type" (ou "Modifier le type d'exécution").
# 3. Dans la section "Hardware accelerator" (ou "Accélérateur matériel"), sélectionnez "GPU".
#
# L'utilisation d'un runtime GPU permet d'accélérer considérablement les calculs, en particulier lors de l'inférence ou de l'entraînement
# de modèles de machine learning. Colab propose des runtimes GPU gratuits avec des quotas limités, mais ces quotas sont souvent suffisants
# pour analyser plusieurs dizaines de milliers de textes. Assurez-vous d'économiser ces ressources pour des tâches nécessitant de fortes
# performances.
#
# Note : Si les quotas GPU sont épuisés ou si vous n'avez pas accès à un GPU, le code fonctionnera tout de même, mais de manière plus lente.

In [1]:
# Montage de Google Drive dans l'environnement Colab afin d'accéder aux fichiers stockés sur le cloud.
# La fonction 'drive.mount()' permet d'établir une connexion avec le Google Drive de l'utilisateur
# et de le rendre accessible sous le chemin '/content/drive/' dans Colab.
#
# 'folder_path' définit le chemin d'accès au dossier spécifique sur Google Drive où les fichiers
# nécessaires à l'analyse seront stockés ou récupérés. Ce chemin sera utilisé pour lire et écrire des fichiers.
# Il est impératif que 'folder_path' se termine par un '/' pour garantir un accès correct aux fichiers dans le dossier.
from google.colab import drive
drive.mount('/content/drive/')
folder_path = '/content/drive/MyDrive/COLAB_NLP/DATA/'  # Assurez-vous que ce chemin se termine par un '/'

Mounted at /content/drive/


In [ ]:
from huggingface_hub import login  # Importation de la fonction 'login' depuis la bibliothèque 'huggingface_hub',
                                   # qui permet de se connecter à un compte Hugging Face et d'accéder aux modèles et ressources.

# Déclarer directement dans le code :
# La fonction 'login' est utilisée ici pour authentifier un utilisateur en utilisant un jeton d'accès (Access Token).
# Note importante : Inclure directement un jeton d'accès dans le code est fortement déconseillé pour des raisons de sécurité.
# Il est préférable d'utiliser des variables d'environnement ou des fichiers de configuration sécurisés pour stocker
# des informations sensibles telles que les jetons d'accès.

login("")  # Authentification avec le jeton d'accès. Remplacez par un jeton sécurisé.

In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from tqdm.notebook import tqdm

def detect_separator(csv_file_path: str) -> str:
    with open(csv_file_path, 'r', encoding='utf-8') as f:
        first_line = f.readline()
    nb_commas = first_line.count(',')
    nb_semicolons = first_line.count(';')
    return ';' if nb_semicolons > nb_commas else ','

def analyze_sentiment(text: str, sentiment_pipeline) -> int:
    result = sentiment_pipeline(text)[0]
    label = result['label']  # ex : '4 stars'
    stars = int(label.split()[0])
    return stars

def main(csv_file_path: str, save_interval: int = 1000):
    full_path = os.path.join(folder_path, csv_file_path)

    # 1. Détecter le séparateur
    sep = detect_separator(full_path)
    print(f"[*] Séparateur détecté : '{sep}'")

    # 2. Lecture du CSV
    df = pd.read_csv(full_path, sep=sep)
    print(f"[*] CSV chargé : {len(df)} lignes.")

    # 3. Colonne 'sentiment'
    if 'sentiment' not in df.columns:
        df['sentiment'] = np.nan
        print("[*] Colonne 'sentiment' inexistante : elle a été créée.")
    else:
        print("[*] Colonne 'sentiment' déjà existante.")
    df['sentiment'] = df['sentiment'].astype('Int64')  # Force un type int nullable

    # 4. Lignes à mettre à jour
    mask_na = df['sentiment'].isna()
    indices_to_update = df[mask_na].index
    print(f"[*] Nombre de lignes à mettre à jour : {len(indices_to_update)}")

    if len(indices_to_update) == 0:
        print("[*] Aucune ligne à mettre à jour. Le script se termine.")
        return

    # 5. Charger le modèle et créer le pipeline
    model_name = 'nlptown/bert-base-multilingual-uncased-sentiment'
    print("[*] Chargement du modèle et du tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    # Choix du device : GPU si disponible, sinon CPU
    device = 0 if torch.cuda.is_available() else -1
    print(f"[*] device={device} (0 = GPU, -1 = CPU)")

    sentiment_pipeline = pipeline(
        'sentiment-analysis',
        model=model,
        tokenizer=tokenizer,
        truncation=True,
        max_length=512,
        device=device
    )
    print("[*] Pipeline d'analyse de sentiments chargé.")

    # 6. Analyser et enregistrer régulièrement
    for i, idx in enumerate(tqdm(indices_to_update, desc="Processing Documents"), start=1):
        text = str(df.at[idx, 'text'])
        if not text.strip():
            df.at[idx, 'sentiment'] = pd.NA
        else:
            stars = analyze_sentiment(text, sentiment_pipeline)
            df.at[idx, 'sentiment'] = stars

        # Sauvegarde intermédiaire
        if i % save_interval == 0:
            df.to_csv(full_path, index=False, sep=sep, encoding='utf-8')
            print(f"[*] Sauvegarde intermédiaire après {i} documents traités.")

    # 7. Sauvegarde finale
    df.to_csv(full_path, index=False, sep=sep, encoding='utf-8')
    print(f"[*] Fichier CSV '{full_path}' mis à jour avec succès.")

if __name__ == '__main__':
    csv_file = 'transcripts.csv'
    main(csv_file, save_interval=1000)